In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
raw_data_path = Path("../data/raw/CPFL-RS-Informações industrial grupo A.xlsx").resolve()

processed_data_folder = Path("../data/processed").resolve()
processed_data_folder.mkdir(parents=True, exist_ok=True)

data = pd.read_excel(raw_data_path)
data.head(30)

In [ ]:
columns_vars = data.iloc[3,0:7].values
columns_months = data.iloc[2,7:19].values
data_columns = np.append(columns_vars, columns_months)

data_consumption = data.iloc[:,0:19]
data_consumption.columns = data_columns
data_consumption = data_consumption.drop([0,1,2,3])
data_consumption = data_consumption.melt(columns_vars).dropna(subset="value")
data_consumption["value"] = pd.to_numeric(data_consumption["value"])

data_demand = pd.concat([data.iloc[:,0:7], data.iloc[:,19:31]], axis=1)
data_demand.columns = data_columns
data_demand = data_demand.drop([0,1,2,3])
data_demand = data_demand.melt(columns_vars).dropna(subset="value")
data_demand["value"] = pd.to_numeric(data_demand["value"])

data_fer = pd.concat([data.iloc[:,0:7], data.iloc[:,31:]], axis=1)
data_fer.columns = data_columns
data_fer = data_fer.drop([0,1,2,3])
data_fer = data_fer.melt(columns_vars).dropna(subset="value")
data_fer["value"] = pd.to_numeric(data_fer["value"])

data_all = pd.concat(
    [data_consumption, data_demand, data_fer],
    axis=0
)

file_save_path = processed_data_folder / raw_data_path.name 
data_all.to_csv(file_save_path, index=False)
